In [ ]:
import pickle
import fairness_metrics
import pandas as pd
from tqdm import tqdm

In [ ]:
# read the data
ideology_df = pd.read_csv('./data/processed_annotated_comments.csv')
ideology_df['label'] = ideology_df['label'].apply(lambda x: None if x not in ['left', 'right'] else x)
ideology_df.dropna(inplace=True)
gender_df = pd.read_csv('./data/jigsaw/gender.csv')
race_df = pd.read_csv('./data/jigsaw/race.csv')
disability_df = pd.read_csv('./data/jigsaw/disability.csv')
sexual_orientation_df = pd.read_csv('./data/jigsaw/sexual_orientation.csv')
dfs = [ideology_df, gender_df, race_df, disability_df, sexual_orientation_df]
names = ['ideology', 'gender', 'race', 'disability', 'sexual orientation']

In [ ]:
# gold standard moderation
with open('./results/comments_moderated_openai.pkl', 'rb') as handle:
    gold1 = pickle.load(handle)

with open('./results/comments_moderated_perspective_nonmanager.pkl', 'rb') as handle:
    gold2 = pickle.load(handle)

with open('./results/comments_moderation_google.pkl', 'rb') as handle:
    gold3 = pickle.load(handle)

with open('./results/comment_moderation_clarifai.pkl', 'rb') as handle:
    gold4 = pickle.load(handle)

In [ ]:
gold1 = utils.process_openai(gold1)
gold2 = utils.process_perspective(gold2)
gold3 = utils.process_google(gold3)
gold4 = utils.process_clarifai(gold4)

gold1 = {k: 0 if v is True else 1 for k,v in gold1.items()}
gold2 = {k: 0 if v is True else 1 for k,v in gold2.items()}
gold3 = {k:0 if v is True else 1 for k,v in gold3.items()}
gold4 = {k:0 if v is True else 1 for k,v in gold4.items()}#{k:0 if max(list(v.values())) >= 0.5 else 1 for k,v in gold4.items()}

In [ ]:
golds = [gold1, gold2, gold3, gold4]

paraphrased = ['./results/comments_backtranslated_german_similarity.pkl', './results/comment_paraphrased_gpt-3.5_final.pkl']
methods = ['openai', 'perspective','google', 'clarifai']
gpt_moderations = [['./results/comments_backtranslated_german_moderated_openai.pkl',
     './results/comments_paraphrased_gpt-3.5_final_moderated_openai.pkl'], 
     ['./results/comments_backtranslated_german_moderation_google_perspective_binary.pkl', 
     './results/comments_moderation_google_perspective_gpt-3.5_final_binary.pkl'],
     ['./results/comments_backtranslated_german_moderation_google_latest_may25_binary.pkl',
     './results/comment_paraphrased_gpt-3.5_final_moderation_google_latest_binary.pkl'],
     ['./results/german_comment_moderation_clarifai.pkl',
     './results/comment_paraphrased_gpt-3.5_final_moderation_clarifai.pkl']]

results = []
global_results = {}

for gold, method, gpt_moderation in zip(golds, methods, gpt_moderations):
    local_gold = gold.copy()
    global_results[method] = {}
    submethods = ['backtranslated', 'paraphrased']
    for augmented, moderated, submethod in zip(paraphrased, gpt_moderation, submethods):
        with open(augmented, 'rb') as handle:
            data = pickle.load(handle)

        with open(moderated, 'rb') as handle:
            mod = pickle.load(handle)

        if 'openai' == method:
            mod = utils.process_openai(mod)
        elif 'perspective' == method:
            mod = utils.process_perspective(mod)
        elif 'google' == method:
            mod = utils.process_google(mod)
        elif 'clarifai' == method:
            mod = utils.process_clarifai(mod)

        if 'backtranslated' == submethod:
            data = {k:v['augmented'] for k,v in data.items() if v['score'] > 0.85 and v['score'] != 1.0}
        
        
        data = {k:1-int(mod[v]) for k,v in data.items() if v in mod.keys()}

        local_gold = {k:v for k,v in local_gold.items() if k in data.keys()}

        # create lists and compute robustness
        global_results[method][submethod] = {}
        for subset, name in zip(dfs, names):
            df = subset.copy()
            df = df[df['text'].isin(list(local_gold.keys()))]
            a = [local_gold[k] for k in df['text'].tolist()]
            b = [data[k] for k in df['text'].tolist()]
            
            r = fairness_metrics.robustness(a, b)
            global_results[method][submethod][name] = r*100
            print(r*100, method, submethod, name)


In [ ]:
global_results